We have obtained the embeddings for 10,000 node networks of params in this folder:

In [1]:
k=5 # k = {5,10,50}
mu = 0.1
run_no = 1

base = f"/nobackup/gogandhi/alt_means_sans_k/data/experiment_n2v_metric_change_10000_{k}_3.0_minc50/Run_{run_no}/" 

net_filename = f"net_LFR_n_10000_tau1_3.0_tau2_1.0_mu_{mu}_k_{k}_mincomm_50.npz"  # A = sp.load_npz(net_path)
comm_filename = f"community_table_LFR_n_10000_tau1_3.0_tau2_1.0_mu_{mu}_k_{k}_mincomm_50.csv" # pd.read_csv()
emb_filename = f"embeddings_LFR_n_10000_tau1_3.0_tau2_1.0_mu_{mu}_k_{k}_mincomm_50.pkl" # embeddings_dict

For instance, we load it to see the embeddings as:

In [2]:
import pickle
with open(base+emb_filename, 'rb') as f:  # open a text file
    emb_dict = pickle.load(f) # deserialize using load()
emb_dict.keys()

dict_keys(['dot', 'euclidean', 'cosine'])

Now we want to take these embeddings, and run k-means clustering with different metrics on them to see which combination comes out on top.
N2V ... K-Means \
Euc ... Dot? \
Euc ... Euc? \
Dot ... Dot? \
I have a strong feeling this is bound to change based on the dimensionality of the embedding vectors, so I will test the cases with embedding dimensions = 8,16,32,128 also. But we're getting ahead of ourselves now.


# The modified K-Means algorithm:

In [15]:
# Define a function that calculates element-centric similarity:
def calc_esim(y, ypred):

    ylab, y = np.unique(y, return_inverse=True)
    ypredlab, ypred = np.unique(ypred, return_inverse=True)
    
    Ka, Kb = len(ylab), len(ypredlab)
    K = np.maximum(Ka, Kb)
    N = len(y)
    
    UA = sparse.csr_matrix((np.ones_like(y), (np.arange(y.size), y)), shape=(N,K))
    UB = sparse.csr_matrix((np.ones_like(ypred), (np.arange(ypred.size), ypred)), shape=(N, K))    
    
    nA = np.array(UA.sum(axis=0)).reshape(-1)
    nB = np.array(UB.sum(axis=0)).reshape(-1)

# nAB[i][j] is read as the number of elements that belong to ith ground truth label and jth predicrted label.
# nAB[1][0] = 1 For ground truth label with index 1 and predicted label 0 we have 1 element. i.e. 0000|1| vs 1110|0|

    nAB = (UA.T @ UB).toarray()
    nAB_rand = np.outer(nA, nB) / N
    
# assuming that each element has an equal probability of being assigned to any label,
# and the expected counts are calculated based on label frequencies.


    # Calc element-centric similarity
    Q = np.maximum(nA[:, None] @ np.ones((1, K)), np.ones((K, 1)) @ nB[None, :]) 
    Q = 1 / np.maximum(Q, 1)
    S = np.sum(np.multiply(Q, (nAB**2))) / N
    
    # Calc the expected element-centric similarity for random partitions
    #Q = np.maximum(nA[:, None] @ np.ones((1, K)), np.ones((K, 1)) @ nB[None, :]) 
    #Q = 1 / np.maximum(Q, 1)
    Srand = np.sum(np.multiply(Q, (nAB_rand**2))) / N
    Scorrected = (S - Srand) / (1 - Srand)
    return Scorrected


In [66]:
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils import check_random_state
from numba import njit
from sklearn.cluster import KMeans 
from scipy import sparse

import pandas as pd
import pickle


class CustomKMeans:
    def __init__(self, n_clusters, metric='euclidean', max_iter=300, tol=1e-4, random_state=None, n_init=20, init='k-means++', batch_size=None):
        self.n_clusters = n_clusters
        self.metric = metric
        self.max_iter = max_iter
        self.tol = tol
        self.random_state = random_state
        self.n_init = n_init
        self.init = init
        self.batch_size = batch_size  # Mini-batch size, if any

    def calculate_distances(self, X, centroids):
        """Optimized distance calculation for the specified metric."""
        if self.metric == 'euclidean':
            return cdist(X, centroids, metric='euclidean')
        elif self.metric == 'manhattan':
            return cdist(X, centroids, metric='cityblock')
        elif self.metric == 'cosine':
            return 1 - cosine_similarity(X, centroids)
        elif self.metric == 'dot':
            return -np.dot(X, centroids.T)
        elif self.metric == 'geodesic':
            # Calculate geodesic distance as arccos(cosine_similarity) for normalized data
            cos_sim = cosine_similarity(X, centroids)
            # Clip values to avoid out-of-domain errors in arccos
            cos_sim = np.clip(cos_sim, -1.0, 1.0)
            return np.arccos(cos_sim)
        else:
            raise ValueError(f"Unsupported metric: {self.metric}")

    def _initialize_centroids(self, X, rng):
        """Efficient k-means++ initialization."""
        centroids = [X[rng.randint(X.shape[0])]]
        closest_dist_sq = self.calculate_distances(X, np.array(centroids))[:, 0] ** 2

        for _ in range(1, self.n_clusters):
            probs = closest_dist_sq / closest_dist_sq.sum()
            cumulative_probs = np.cumsum(probs)
            r = rng.rand()
            new_centroid = X[np.searchsorted(cumulative_probs, r)]
            centroids.append(new_centroid)
            new_dist_sq = self.calculate_distances(X, np.array([new_centroid]))[:, 0] ** 2
            closest_dist_sq = np.minimum(closest_dist_sq, new_dist_sq)
        
        return np.array(centroids)

    @staticmethod
    @njit
    def _update_centroids(X, labels, n_clusters):
        """Compute new centroids using JIT compilation for efficiency."""
        new_centroids = np.zeros((n_clusters, X.shape[1]), dtype=X.dtype)
        counts = np.zeros(n_clusters, dtype=np.int64)
        
        for i in range(X.shape[0]):
            new_centroids[labels[i]] += X[i]
            counts[labels[i]] += 1
        
        for j in range(n_clusters):
            if counts[j] > 0:
                new_centroids[j] /= counts[j]
        
        return new_centroids

    def _run_kmeans(self, X, rng):
        """Run a single instance of K-means clustering with optional mini-batch."""
        centroids = self._initialize_centroids(X, rng)
        n_samples = X.shape[0]

        for i in range(self.max_iter):
            if self.batch_size:
                batch_indices = rng.choice(n_samples, self.batch_size, replace=False)
                X_batch = X[batch_indices]
                distances = self.calculate_distances(X_batch, centroids)
                labels = np.argmin(distances, axis=1)
            else:
                distances = self.calculate_distances(X, centroids)
                labels = np.argmin(distances, axis=1)
            
            new_centroids = self._update_centroids(X, labels, self.n_clusters)
            
            # Convergence check based on relative tolerance
            centroid_shifts = np.linalg.norm(new_centroids - centroids, axis=1)
            if np.all(centroid_shifts < self.tol * np.linalg.norm(centroids, axis=1)):
                break
            
            centroids = new_centroids
        
        # Inertia calculation for this run
        inertia = np.sum(np.min(distances, axis=1) ** 2)
        return centroids, labels, inertia

    def fit(self, X):
        """Run KMeans with multiple initializations to get the best clustering."""
        best_inertia = np.inf
        best_centroids = None
        best_labels = None
        rng = check_random_state(self.random_state)

        for _ in range(self.n_init):
            centroids, labels, inertia = self._run_kmeans(X, rng)
            
            if inertia < best_inertia:
                best_inertia = inertia
                best_centroids = centroids
                best_labels = labels
        
        # Set final results
        self.centroids_ = best_centroids
        self.labels_ = best_labels
        self.inertia_ = best_inertia
        return self

    def predict(self, X):
        """Predict the closest cluster each sample in X belongs to."""
        distances = self.calculate_distances(X, self.centroids_)
        return np.argmin(distances, axis=1)


In [71]:
def clustering_method_values(net, community_table, emb, score_keys):
    # Normalize the vector of each node to have unit length. This normalization improves clustering.
    #X = np.einsum("ij,i->ij", emb, 1 / np.maximum(np.linalg.norm(emb, axis=1), 1e-24))
    #X = emb.copy()

    def method_score(key): 
        if key == "kmeans++":
            kmeans = KMeans(n_clusters=len(set(community_table["community_id"])), init='k-means++').fit(emb)
            return calc_esim(community_table["community_id"], kmeans.labels_)

        elif key.startswith("kmeans_"):  # Parse metric and apply CustomKMeans with modified metrics
            metric = key.split("_", 1)[1]
            custom_kmeans = CustomKMeans(n_clusters=len(set(community_table["community_id"])), metric=metric).fit(emb)
            return calc_esim(community_table["community_id"], custom_kmeans.labels_)

    
    # Calculate and store scores for each clustering method in score_keys
    score_dictionary = {}
    for key in score_keys:
        score_dictionary[key] = method_score(key)

    return score_dictionary

Before we parallelize and get results for all the LFR networks of varying mixing rates and varying network densities. We will start with an example of just one network. We load the network, community information, embeddings, and run the modified K-Means using Dot, Euclidean and Cosine similarities on the embedding vectors generated using Node2Vec using Dot, Euclidean and Cosine similarities.

In [68]:
def load_net_and_embedding(net_filename, comm_filename, emb_filename):
    net = sparse.load_npz(net_filename)
    community_table = pd.read_csv(comm_filename)
    
    with open(emb_filename, 'rb') as f:  # open a text file
        emb_dict = pickle.load(f) # deserialize using load()

    return net, community_table, emb_dict

In [69]:
N=10000
mu_values = np.round(np.arange(0.05, 1.05, 0.05),decimals=2)

params = {
    "N": N,
    "k": 5,
    "maxk":  int(np.sqrt(10 * N)),
    "minc": 50,
    "maxc": int(np.ceil(np.sqrt(N * 10))),
    "tau": 3.0,
    "tau2": 1.0,
    "mu": 0.2,
    }


emb_params = {
    "method": "node2vec",
    "window_length": 10,
    "walk_length": 80,
    "num_walks": 10,
    "dim": 64,
}


k=5 # k = {5,10,50}
mu = 0.1
run_no = 1

path_name = f"/nobackup/gogandhi/alt_means_sans_k/data/experiment_n2v_metric_change_10000_{k}_3.0_minc50/Run_{run_no}/" 

net_filename = path_name + f"net_LFR_n_10000_tau1_3.0_tau2_1.0_mu_{mu}_k_{k}_mincomm_50.npz"  # A = sp.load_npz(net_path)
comm_filename = path_name + f"community_table_LFR_n_10000_tau1_3.0_tau2_1.0_mu_{mu}_k_{k}_mincomm_50.csv" # pd.read_csv()
emb_filename = path_name + f"embeddings_LFR_n_10000_tau1_3.0_tau2_1.0_mu_{mu}_k_{k}_mincomm_50.pkl" # embeddings_dict

#"community_table_LFR_n_10000_tau1_3.0_tau2_1.0_mu_0.1_k_50_mincomm_50.npz"

net, community_table, emb_dict = load_net_and_embedding(net_filename, comm_filename, emb_filename)

In [74]:
score_keys=['kmeans++','kmeans_euclidean','kmeans_dot','kmeans_cosine'] 
for key in ['dot', 'euclidean', 'cosine']:
    emb = emb_dict[key]
    print(clustering_method_values(net, community_table, emb, score_keys))

{'kmeans++': 0.8883255562176959, 'kmeans_euclidean': 0.9190984190926753, 'kmeans_dot': 0.676045991603713, 'kmeans_cosine': 0.8216621800325635}
{'kmeans++': 0.9925325345751824, 'kmeans_euclidean': 0.9925325345751824, 'kmeans_dot': 0.6464478510220129, 'kmeans_cosine': 0.9903730792768748}
{'kmeans++': 0.9478108231369303, 'kmeans_euclidean': 0.9385970563145652, 'kmeans_dot': 0.7824215655820644, 'kmeans_cosine': 0.9515917299553007}


We go through 20 iterations of Kmeans, and use the best clustering of the them. The Kmeans++ (which is the standard optimized implementation) performs comparable to our Kmeans_euclidean which is our baseline for our modified version especially in the euclidean-euclidean case. When embeddings are generated using other methods, it does falter a bit. This is exciting news! Which means there could be more to uncover!

# Parallelization to get clustering

In [77]:
import os
import numpy as np
import pandas as pd
import time
from concurrent.futures import ProcessPoolExecutor, as_completed
import torch

# Assuming these functions are defined elsewhere
# from your_module import load_net_and_embedding, clustering_method_values

# Parameters
N = 10000
K = 50
mu_values = np.round(np.arange(0.05, 1.05, 0.05), decimals=2)
params_template = {
    "N": N,
    "k": K,
    "maxk": int(np.sqrt(10 * N)),
    "minc": 50,
    "maxc": int(np.ceil(np.sqrt(N * 10))),
    "tau": 3.0,
    "tau2": 1.0,
}
emb_params = {
    "method": "node2vec",
    "window_length": 10,
    "walk_length": 80,
    "num_walks": 10,
    "dim": 64,
}

score_keys = ['kmeans++', 'kmeans_euclidean', 'kmeans_dot', 'kmeans_cosine']

# Output directory – we will create one file per embedding type
output_dir = f"/nobackup/gogandhi/alt_means_sans_k/data/experiment_n2v_metric_change_10000_{K}_3.0_minc50/"


# Function to process a single run and mu value
def process_run(run_no, mu):
    # Build file paths based on run and mu
    
    path_name = f"/nobackup/gogandhi/alt_means_sans_k/data/experiment_n2v_metric_change_10000_{K}_3.0_minc50/Run_{run_no}/"
    net_filename = path_name + f"net_LFR_n_10000_tau1_3.0_tau2_1.0_mu_{mu}_k_{K}_mincomm_50.npz"
    comm_filename = path_name + f"community_table_LFR_n_10000_tau1_3.0_tau2_1.0_mu_{mu}_k_{K}_mincomm_50.csv"
    emb_filename = path_name + f"embeddings_LFR_n_10000_tau1_3.0_tau2_1.0_mu_{mu}_k_{K}_mincomm_50.pkl"
    
    # Load network, community table, and the embedding dictionary
    net, community_table, emb_dict = load_net_and_embedding(net_filename, comm_filename, emb_filename)
    
    # For each embedding in the dictionary, run clustering and prepare a result string
    results = []
    for emb_key, emb in emb_dict.items():
        result = clustering_method_values(net, community_table, emb, score_keys)
        result_values = [result[key] for key in score_keys]
        # Format: run_no,mu,score1,score2,...
        result_str = f"{run_no},{mu}," + ",".join(map(str, result_values))
        results.append((emb_key, result_str))
        print(f"Completed Run {run_no} with Mu {mu} for embedding '{emb_key}'")
    return results

# Function to process all run/mu combinations in parallel
def process_all_combinations_parallel():
    # Generate all combinations of run numbers and mu values
    runs_mu_combinations = [(run_no, mu) for run_no in range(1, 2) for mu in [0.1]]
    total_combinations = len(runs_mu_combinations)
    start_time = time.time()
    #print(total_combinations)
    # Dictionary to collect results per embedding type
    all_results = {}  # {embedding_key: [result_str, ...]}
    
    # Parallel processing using ProcessPoolExecutor
    with ProcessPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(process_run, run_no, mu): (run_no, mu) 
                   for run_no, mu in runs_mu_combinations}
        
        # As each future completes, collect its results
        for idx, future in enumerate(as_completed(futures), start=1):
            run_no, mu = futures[future]
            try:
                run_results = future.result()  # List of (emb_key, result_str) tuples
                for emb_key, result_str in run_results:
                    if emb_key not in all_results:
                        all_results[emb_key] = []
                    all_results[emb_key].append(result_str)
                print(f"Processed {idx}/{total_combinations} combinations: Run {run_no}, Mu {mu}")
            except Exception as e:
                print(f"Error processing Run {run_no}, Mu {mu}: {e}")
    
    # Write out separate output files per embedding type
    for emb_key, results in all_results.items():
        output_file = os.path.join(output_dir, f"n2v_{emb_key}_kmeans_clustering.txt")
        with open(output_file, "w") as f:
            header = "run_no,mu," + ",".join(score_keys) + "\n"
            f.write(header)
            for line in results:
                f.write(line + "\n")
        print(f"Results written to {output_file}")
    
    total_elapsed_time = time.time() - start_time
    print(f"All combinations processed in parallel. Total elapsed time: {total_elapsed_time:.2f} seconds.")

if __name__ == "__main__":
    process_all_combinations_parallel()


1


Process ForkProcess-23:
Process ForkProcess-24:
Process ForkProcess-29:
Process ForkProcess-28:
Process ForkProcess-30:
Process ForkProcess-22:
Process ForkProcess-26:
Process ForkProcess-25:
Process ForkProcess-27:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/nobackup/gogandhi/miniconda3/envs/gensim_mod_env/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/nobackup/gogandhi/miniconda3/envs/gensim_mod_env/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/nobackup/gogandhi/miniconda3/envs/gensim_mod_env/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/nobackup/gogandhi/miniconda3/envs/gensim_mod_env/lib/python3.9/multiprocess

KeyboardInterrupt: 

In [79]:
# Instead of using ProcessPoolExecutor, run the function directly:

import time
start = time.time()
results = process_run(1, 0.1)
print("Direct call results:", results)
print(time.time()-start)

Completed Run 1 with Mu 0.1 for embedding 'dot'
Completed Run 1 with Mu 0.1 for embedding 'euclidean'
Completed Run 1 with Mu 0.1 for embedding 'cosine'
Direct call results: [('dot', '1,0.1,0.930774835825718,0.9078791558988919,0.8109024487542578,0.8850640546357088'), ('euclidean', '1,0.1,1.0,1.0,0.47745189633168655,1.0'), ('cosine', '1,0.1,0.9577367930235333,0.9694957803036819,0.7632839896711154,1.0')]
72.08514332771301


In [2]:
import torch 
import gc

# Invoke garbage collector
gc.collect()

# Clear GPU cache
torch.cuda.empty_cache()